In [1]:
import tensorflow as tf
import numpy as np

In [2]:
from numpy.random import RandomState
rdm = RandomState(1)

y_  = tf.constant(rdm.rand(1,10))
y = np.array([0.0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0])

cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=y_)
with tf.Session() as sess:
    loss = sess.run(cross_entropy)
    print(loss)

y_softmax = tf.nn.softmax(logits=y_)
cross_entropy_v1 = -tf.reduce_sum(y * tf.log(y_softmax))

with tf.Session() as sess:
    loss = sess.run(cross_entropy_v1)
    print(loss)

[1.91865955]
1.9186595478618151


In [3]:
logits = tf.constant([[1.0,2.0,3.0],[1.0,2.0,3.0],[1.0,2.0,3.0]])
y  = tf.nn.softmax(logits)
y_ = tf.constant([[0.0,0.0,1.0],[0.0,0.0,1.0],[0.0,0.0,1.0]])
cross_entropy = -tf.reduce_sum(y_*tf.log(y))

cross_entropy2 = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_))

with tf.Session() as sess:
    c_e = sess.run(cross_entropy)
    c_e2 = sess.run(cross_entropy2)
    print("step2:cross_entropy result=")
    print(c_e)
    print("Function(softmax_cross_entropy_with_logits) result=")
    print(c_e2)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

step2:cross_entropy result=
1.222818
Function(softmax_cross_entropy_with_logits) result=
1.2228179


In [4]:
Truth = np.array([0,0,1,0])
Pred_logits = np.array([3.5,2.1,7.89,4.4])

loss = tf.nn.softmax_cross_entropy_with_logits(labels=Truth,logits=Pred_logits)
loss2 = tf.nn.softmax_cross_entropy_with_logits_v2(labels=Truth,logits=Pred_logits)
loss3 = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=tf.argmax(Truth),logits=Pred_logits)

with tf.Session() as sess:
    print(sess.run(loss))
    print(sess.run(loss2))
    print(sess.run(loss3))

0.04493472585997595
0.04493472585997595
0.04493472585997595
